# Mistral Quantization and Finetuning

In [1]:
!pip install -q accelerate
!pip install -qi https://pypi.org/simple/ bitsandbytes

In [ ]:
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM

In [ ]:
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

## Quantization
Quantization is done reduce memory footprint and perform faster inference while still retaining acceptable model performance. For this quantization, we will use bitandbytes

In [ ]:
# Use 4 bit compute 
use_4bit = True

# compute dtype for 4-bit models
compute_dtype = "float16"

# quantization type
quantization_dtype = 'nf4'

# use double quantization
use_nested_quant = False

### Quantization config

In [ ]:
bnb_compute_dtype = getattr(torch, compute_dtype)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=quantization_dtype,
    bnb_4bit_compute_dtype=bnb_compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
)

In [ ]:
chat_input = tokenizer.encode_plus("[INST] Was Vivek Ramaswamy running for president ? [/INST]", return_tensors="pt")['input_ids'].to('cuda')

generated_ids = model.generate(chat_input, 
                               max_new_tokens=1000, 
                               do_sample=True, 
                               pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)